# MapReduce

The goal of this lab is to give you experience thinking in terms of MapReduce. We will be using small datasets that you can inspect manually to determine the correctness of your results to help you internalize how MapReduce works. In the next lab, you will have the opportunity to use Spark, a MapReduce-based system, to process the very large datasets for which it was actually designed.


## Learning the Basics

We will first look at the map() and reduce() functions individually and then we will use them together to build more complex exercises.


### The map() Function

First, let's think in terms of the map() function.  The map() function iterates through all the items in the given iterable and executes the function we passed as an argument on each of them. If you're asking yourself "how is this different from a regular loop?" the answer is it's obvious how to parallelize a map function without any further input from the programmer. 

Consider for example a list of fruits like the one below:

In [1]:
fruits = ["Apple", "Strawberry", "Banana", "Pear", "Apricot", "Watermelon", "Orange", "Avocado", "Pineapple"]

How can you take the list of fruits and get another list of which ones begin with the letter "A"?

In [2]:
# Option 1: Defining your own begins_with_A function
def begins_with_A(word):
    return word[0] == "A"

bool_fruits_A = list(map(begins_with_A, fruits))
print(bool_fruits_A)

[True, False, False, False, True, False, False, True, False]


In [3]:
# Option 2: A nicer and more compact way
bool_fruits_A = list(map(lambda s: s[0] == "A", fruits))
print(bool_fruits_A)

[True, False, False, False, True, False, False, True, False]


### The reduce() Function

Now that you know how to use map(), let's give reduce() a try.  First, remember that reduce() returns a single value based on the function and iterable we've passed (instead of an iterator). Second, reduce() takes a function that is commutative and associative, that is, the order of the elements does not affect the result and the grouping of the elements also does not affect the result. This means you need to map the data into the same type of result you're expecting after applyting reduce().

Also note that in Python 3 reduce() isn't a built-in function anymore, but it can be found in the functools module.


In [4]:
from functools import reduce

some_list = [2, 4, 7, 3, 1, 10, 21, 42]
print(reduce(lambda x, y: x + y, some_list))

90


Sum is commutative and associative, so any splitting and reordering of <code>some_list</code> will yild the same result:

In [26]:
half_some_list_1 = [2, 4, 7, 3]
half_some_list_2 = reversed([1, 10, 21, 42])

reduced_1 = reduce(lambda x, y: x + y, half_some_list_1)
reduced_2 = reduce(lambda x, y: x + y, half_some_list_2)

print(reduce(lambda x, y: x + y, [reduced_1,reduced_2]))

90


This is important again because, when working with very large datasets, your data will be split and you want to get correct results when doing things in parallel.

**Mini exercise**: use map() and reduce() to count how many fruits begin with the letter "A". Use the cell below to try it out!


<details>
<summary>
<font size="3" color="green">
<b>Click here to see one possible solution.</b>
</font>
</summary>
<code>int_fruits_A = map(lambda s: int(s),bool_fruits_A)
print(reduce(lambda x, y: x + y, int_fruits_A))
</code>
</details>

In [27]:
# Use this cell to type your answer or copy and paste the answer hidden above
int_fruits_A = map(lambda s: int(s),bool_fruits_A)
print(reduce(lambda x, y: x + y, int_fruits_A))

3


## Warm Up Exercise: A Social Network

Now that you know how to use map and reduce, let's practice doing more complex things.

Consider a simple social network dataset consisting of a set of key-value pairs (person, friend) representing a friend relationship between two people. 

Each input record is a pair person_A, person_B where person_A is a string representing the name of a person and person_B is a string representing the name of one of person_A's friends. Note that it may or may not be the case that the relationship is symmetric, that is, person_B might not consider person_A a friend. 

**Task**: Describe a MapReduce algorithm to count the number of friends for each person. The output should be a pair (person, friend_count) where person is a string and friend_count is an integer indicating the number of friends associated with person.


### Map the Input 

Let's begin by reading the input file "friends.dat".

In [6]:
#import ast

# Loading the data
network_data_file = open("friends.dat")
network_data = network_data_file.read().split('\n')
network_data_file.close()

You can open the file or print 'network_data' to see how the data looks like.

In [9]:
print(network_data)

['Myriel, Geborand', 'Myriel, Champtercier', 'Myriel, Count', 'Myriel, OldMan', 'Myriel, Valjean', 'Napoleon, Myriel', 'MlleBaptistine, Myriel', 'MlleBaptistine, Valjean', 'MlleBaptistine, MmeMagloire', 'MmeMagloire, Myriel', 'MmeMagloire, Maria', 'Champtercier, Myriel', 'Valjean, Myriel', 'Valjean, MmeMagloire', 'Valjean, Labarre', 'Valjean, Marguerite', 'Valjean, MmeDeR', 'Valjean, Isabeau', 'Valjean, Fantine', 'Valjean, Cosette', 'Valjean, Simplice', 'Valjean, Woman1', 'Valjean, Judge', 'Valjean, Woman2', 'Valjean, Gillenormand', 'Valjean, MlleGillenormand', 'Valjean, Babet', 'Valjean, Montparnasse', 'Maria, Carlos', 'Maria, Ana', 'Carlos, Ana', 'Carlos, Maria', 'Ana, Maria', 'Enola, Alba', 'Enola, Ebba', 'Enola, Maria', 'Enola, Juana', 'Enola, Sherlock', 'Enola, John', 'Monica, Rachel']


Now that the data has been loaded, think about the format you'll need to map your data into. In order to reduce your data into a list of pairs of the form <code>(person, num_friends)</code>, you'll have to map the original input data into something similar.

In [10]:
person_1_pairs = list(map(lambda p: [((p.split(','))[0],1)], network_data))

print(person_1_pairs)

[[('Myriel', 1)], [('Myriel', 1)], [('Myriel', 1)], [('Myriel', 1)], [('Myriel', 1)], [('Napoleon', 1)], [('MlleBaptistine', 1)], [('MlleBaptistine', 1)], [('MlleBaptistine', 1)], [('MmeMagloire', 1)], [('MmeMagloire', 1)], [('Champtercier', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Valjean', 1)], [('Maria', 1)], [('Maria', 1)], [('Carlos', 1)], [('Carlos', 1)], [('Ana', 1)], [('Enola', 1)], [('Enola', 1)], [('Enola', 1)], [('Enola', 1)], [('Enola', 1)], [('Enola', 1)], [('Monica', 1)]]


Now that we mapped our data, it's almost time to use reduce(). The reduceByKey() function is only available in pyspark, but we can still write our own reduce_by_key function that is commutative and associative and use it together with reduce().

In [7]:
## We consider 2 lists because the function should be conmmutative and assosiative. 
## Keeping the lists sorted reduces the time complexity of the function, 
## which is not important for tiny problems like this one, but it's very important for large datasets
def reduce_by_key(list1,list2):
    final_list = []
    i, j = 0, 0
  
    while i < len(list1) and j < len(list2): 
        if list1[i][0] == list2[j][0]: 
            final_list.append((list1[i][0],list1[i][1] + list2[j][1]))
            i += 1
            j += 1
            
        elif list1[i][0] < list2[j][0]: 
            final_list.append(list1[i]) 
            i += 1
  
        else: 
            final_list.append(list2[j]) 
            j += 1
    
    return list(final_list + list1[i:] + list2[j:])

Now we apply reduce() to get our final count:

In [8]:
person_friends_pairs = list(reduce(reduce_by_key, person_1_pairs))

print(person_friends_pairs)

[('Ana', 1), ('Carlos', 2), ('Champtercier', 1), ('Enola', 6), ('Maria', 2), ('MlleBaptistine', 3), ('MmeMagloire', 2), ('Monica', 1), ('Myriel', 5), ('Napoleon', 1), ('Valjean', 16)]


Can you think of ways of counting the original input data and the number of friends in order to partially validate the output?

<details>
<summary>
<font size="3" color="green">
<b>Click here to see one possible solution.</b>
</font>
</summary>
<code>print(reduce(lambda p , q: ('total',p[1]+q[1]), person_friends_pairs))</code>
</details>

<details>
<summary>
<font size="3" color="green">
<b>Click here to see another possible solution.</b>
</font>
</summary>
<code>friend_count_map = map(lambda p: p[1], person_friends_pairs)
print(reduce(lambda p,q: p+q, friend_count_map))
</code>
</details>

In [44]:
# Use this cell to test your MapReduce answer and compare it with the input

print(len(network_data))

40


## Exercise

Using the same fruit list from the begining of the lab, count how many fruits begin with each letter.

In [ ]:
#Use this cell for your code


## Real Exercise

A more challenging problem about books!